<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|
---
`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|
---
`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [1]:
-- What is the oldest business on each continent?

WITH all_business AS(
	SELECT business, year_founded, country_code
	FROM businesses	b
	UNION ALL 
	SELECT business, year_founded, country_code	
	FROM new_businesses nb
),
oldest_business_continent AS (
	SELECT 
	ab.business, 
    ab.year_founded, 
    ab.country_code,
	c.country,
	c.continent
	FROM all_business AS ab
	RIGHT JOIN countries AS c
	ON ab.country_code = c.country_code
),
min_year AS (
	SELECT obc.continent, MIN(year_founded) AS year_founded
	FROM oldest_business_continent obc
	GROUP BY continent
)	
SELECT 
	obc.continent AS continent,
	obc.country AS country,
	obc.business AS business,
	obc.year_founded AS year_founded
FROM oldest_business_continent obc
JOIN min_year m
ON obc.continent = m.continent AND obc.year_founded = m.year_founded;




,continent,country,business,year_founded
0,Africa,Mauritius,Mauritius Post,1772
1,Asia,Japan,Kongō Gumi,578
2,Europe,Austria,St. Peter Stifts Kulinarium,803
3,North America,Mexico,La Casa de Moneda de México,1534
4,South America,Peru,Casa Nacional de Moneda,1565
5,Oceania,Australia,Australia Post,1809


In [2]:

-- How many countries per continent lack data on the oldest businesses?

WITH all_business AS (
    SELECT business, year_founded, country_code
    FROM businesses
),
location AS (
    SELECT c.country_code, c.continent
    FROM countries c
    LEFT JOIN all_business ab ON c.country_code = ab.country_code
    WHERE ab.year_founded IS NULL
)
SELECT continent, COUNT(DISTINCT country_code) AS countries_without_businesses
FROM location
GROUP BY continent;

-- Does including the `new_businesses` data change this?

WITH all_business AS (
    SELECT business, year_founded, country_code
    FROM businesses
    UNION ALL
    SELECT business, year_founded, country_code
    FROM new_businesses
),
location AS (
    SELECT c.country_code, c.continent
    FROM countries c
    LEFT JOIN all_business ab ON c.country_code = ab.country_code
    WHERE ab.year_founded IS NULL
)
SELECT continent, COUNT(DISTINCT country_code) AS countries_without_businesses
FROM location
GROUP BY continent;

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [3]:


WITH all_business AS (
	SELECT category_code, year_founded, country_code
	FROM businesses
	UNION ALL 
	SELECT category_code, year_founded, country_code	
	FROM new_businesses
),
location AS (
	SELECT DISTINCT continent, category_code, year_founded
	FROM countries c
	JOIN all_business ab
	ON ab.country_code = c.country_code
),
oldest_by_continent_category AS (
	SELECT continent, ct.category, MIN(year_founded) AS year_founded
	FROM location l
	JOIN categories ct 
	ON l.category_code = ct.category_code
	GROUP BY continent, ct.category
)
SELECT DISTINCT continent, category, year_founded
FROM oldest_by_continent_category obcc;



,continent,category,year_founded
0,Asia,Tourism & Hotels,1584
1,Africa,Media,1943
2,Asia,Defense,1808
3,Europe,Defense,1878
4,Europe,"Cafés, Restaurants & Bars",803
5,Africa,"Distillers, Vintners, & Breweries",1933
6,Africa,Aviation & Transport,1854
7,Europe,Manufacturing & Production,864
8,Africa,Agriculture,1947
9,Oceania,Banking & Finance,1861
